In [6]:
# Importações
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import confusion_matrix
from itertools import combinations
import matplotlib.pyplot as plt
import dask.dataframe as dd
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [7]:
# Defina a variável de ambiente OMP_NUM_THREADS para 1
os.environ['OMP_NUM_THREADS'] = '8'

In [8]:
# Leitura do arquivo CSV em partes (chunks) para otimização de uso de memória
csv_file_path = "DrDoS_UDP.csv"
dados = []
chunk_size = 100000

In [9]:
# Carregue seus dados (substitua 'seus_dados.csv' pelo nome do seu arquivo)
dados = pd.read_csv(csv_file_path, low_memory=False)

In [10]:
dados = pd.DataFrame(dados)
len(dados)

3136802

In [11]:
amostra = dados.sample(n=chunk_size, random_state=42)
print(amostra)

         Unnamed: 0                                 Flow ID   Source IP  \
1700177       42873  172.16.0.5-192.168.50.1-53109-38802-17  172.16.0.5   
122791        24068  172.16.0.5-192.168.50.1-48599-42819-17  172.16.0.5   
926756        16865  172.16.0.5-192.168.50.1-52218-13616-17  172.16.0.5   
3007834       44625  172.16.0.5-192.168.50.1-54191-24415-17  172.16.0.5   
748966        59700  172.16.0.5-192.168.50.1-51599-59509-17  172.16.0.5   
...             ...                                     ...         ...   
1684081       84305  172.16.0.5-192.168.50.1-43719-45899-17  172.16.0.5   
2786522       36552  172.16.0.5-192.168.50.1-48524-19399-17  172.16.0.5   
2639910       98707  172.16.0.5-192.168.50.1-56665-60980-17  172.16.0.5   
1725519       19125  172.16.0.5-192.168.50.1-40891-19395-17  172.16.0.5   
2782138       95839  172.16.0.5-192.168.50.1-48627-19483-17  172.16.0.5   

          Source Port  Destination IP   Destination Port   Protocol  \
1700177         53109    192

In [12]:
dados_tr = amostra
print(dados_tr.columns.str.strip())

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [13]:
# Lista das colunas a serem selecionadas
colunas_selecionadas = ['Fwd Packet Length Std', 'Packet Length Std', 'Fwd IAT Mean', 'min_seg_size_forward',
                        'Fwd IAT Max', 'Destination Port', 'Fwd Packet Length Max', 'Fwd Packet Length Mean',
                        'Fwd Packet Length Min', 'Protocol', 'Timestamp']

# Remova espaços em branco dos nomes das colunas
dados_tr.columns = dados_tr.columns.str.strip()

# Verifique se todas as colunas selecionadas estão no DataFrame
if set(colunas_selecionadas).issubset(dados_tr.columns):
    # Selecionar apenas as colunas desejadas
    df_dados = dados_tr[colunas_selecionadas]

    # Mostrar todas as colunas disponíveis no novo DataFrame
    print("Colunas Selecionadas:")
    print(df_dados.columns)
    print(df_dados)
else:
    print(dados_tr.columns)
    print(dados_tr)
    print("Nomes das colunas selecionadas não correspondem às colunas no DataFrame.")

Colunas Selecionadas:
Index(['Fwd Packet Length Std', 'Packet Length Std', 'Fwd IAT Mean',
       'min_seg_size_forward', 'Fwd IAT Max', 'Destination Port',
       'Fwd Packet Length Max', 'Fwd Packet Length Mean',
       'Fwd Packet Length Min', 'Protocol', 'Timestamp'],
      dtype='object')
         Fwd Packet Length Std  Packet Length Std  Fwd IAT Mean  \
1700177                0.00000           0.000000           1.0   
122791                 0.00000           0.000000           1.0   
926756                 0.00000           0.000000          46.0   
3007834                0.00000           0.000000           3.0   
748966                35.08846          33.617597       43443.2   
...                        ...                ...           ...   
1684081                0.00000           0.000000           0.0   
2786522                0.00000           0.000000           1.0   
2639910                0.00000           0.000000           1.0   
1725519                0.00000     

In [14]:
df_dados = df_dados.sort_values(by='Timestamp')

In [15]:
# Função para criar histograma de uma coluna
def criar_histograma(coluna):
    plt.figure(figsize=(8, 6))  # Ajuste o tamanho da figura conforme necessário
    plt.hist(df_dados['Timestamp'], bins=50, alpha=0.5, label='Timestamp')  # Crie o histograma para "Timestamp"
    plt.hist(df_dados[coluna], bins=50, alpha=0.5, label=coluna)  # Crie o histograma para a coluna atual
    plt.xlabel('Timestamp')  # Rótulo do eixo x
    plt.ylabel('Frequência')  # Rótulo do eixo y
    plt.title(f'Histograma para {coluna} com base em Timestamp')  # Título do gráfico
    plt.legend(loc='upper right')  # Adicione uma legenda
    plt.show()  # Mostre o gráfico

In [ ]:
# Crie threads para processar cada coluna
with ThreadPoolExecutor() as executor:
    for coluna in colunas_selecionadas:
        executor.submit(criar_histograma, coluna)
        print(coluna)

Fwd Packet Length Std
Packet Length Std
Fwd IAT Mean
min_seg_size_forward
Fwd IAT Max
Destination Port
Fwd Packet Length Max
Fwd Packet Length Mean
Fwd Packet Length Min
Protocol
Timestamp
